# Load the Raw Data

In [ ]:
%%bash
wget https://raw.githubusercontent.com/eladsegal/strategyqa/refs/heads/main/data/strategyqa/train.json

In [ ]:
import pandas as pd

raw_data_path='/kaggle/working/train.json'

df = pd.read_json(raw_data_path)
df.head(5)

In [ ]:
df.shape

In [ ]:
df_shuffle=df.sample(frac=1).reset_index(drop=True).head(1200)
df_shuffle.head(5)

In [ ]:
df_shuffle.shape

In [ ]:
first=df_shuffle.iloc[0]

print(first['question'])
print(first['facts'])
print(first['evidence'])
print(first['answer'])

# Creating Multiple Reasoning Paths

In [ ]:
from typing import Optional, List
from pydantic import BaseModel

class Entity(BaseModel):
    qid: str
    term: str
    description: str
    question: str
    answer: bool
    facts: list
    decomposition: list
    evidence: list

class ReasoningPath(BaseModel):
    question: str
    evidence: List[str] # for original facts
    reason: str
    answer: str
    ground_truth: bool
    temperature: float
    tokens: int
    score: float

entities = [Entity.model_validate(row) for row in df_shuffle.to_dict(orient='records')]
print(len(entities))
print(entities[0])

In [ ]:
import re
from sot.sot import SoT
from typing import Tuple, Union
from doraemon.logger_util import get_logger
from doraemon.inference_factory import InferenceFactory

logger=get_logger(name=__name__, logfile="strategy_qa_dataset_builder.log")

def get_answer(raw_answer: str)-> str:
    answer=re.search(r"\\boxed\{(.*?)\}", raw_answer)
    if answer:
        return answer.group(1)
    return None
    

def process_entity(args)-> Optional[Tuple[ReasoningPath, int]]:
    """
    """
    entity,paradigm,temperature=args

    try:
        prompt=SoT.get_initialized_prompt(paradigm=paradigm, question=f"Evidences:{entity.facts}\nQuestion:{entity.question}")
        r_s, tokens=InferenceFactory.inference(logger=logger, messages=prompt, temperature=temperature)
        result=ReasoningPath(
            question=str(entity.question), 
            evidence=entity.facts, 
            reason=str(r_s), 
            answer=get_answer(r_s),
            ground_truth=bool(entity.answer),
            temperature=float(temperature), 
            tokens=int(tokens), 
            score=0.0)
        return result, tokens
    except Exception as e:
        logger.error(f"Error processing quetion {entity.question} at temperature {temperature} with exception {e}")
        return None


paradigm = SoT.classify_question(entities[0].question)
logger.info(paradigm)
assert "conceptual_chaining"==str(paradigm)

In [ ]:
temperatures = [i * 0.25 for i in range(9)]  # [0.0, 0.25, 0.5, ... ,2.0]

tasks=[]
for et in entities:
    for tp in temperatures:
        tasks.append((et,paradigm,tp))
tasks[0]

In [ ]:
from tqdm import tqdm
import concurrent.futures

with concurrent.futures.ProcessPoolExecutor(max_workers=1) as executor:
    results=list(tqdm(executor.map(process_entity, tasks), total=len(tasks)))

In [ ]:
from typing import List

multipleReasonPaths: List[ReasoningPath]=[]

for rs_token in results:
    if rs_token is not None:
        reason_path,_=rs_token # tuple
        multipleReasonPaths.append(reason_path)

len(multipleReasonPaths)

In [ ]:
logger.info(multipleReasonPaths[0])
logger.info(multipleReasonPaths[0].answer)

In [13]:
import pickle
from datetime import datetime
from datasets import Dataset

def convert_rp_to_ds(multipleReasonPaths: List[ReasoningPath]) -> Dataset:
    data_dicts = [rp.model_dump() for rp in multipleReasonPaths]
    # Create a Dataset from the list of dictionaries
    return Dataset.from_list(data_dicts)

def to_pkl(ds: Dataset, filename: str):
    with open(filename, 'wb') as f:
        pickle.dump(ds, f)


ds=convert_rp_to_ds(multipleReasonPaths)

to_pkl(ds, filename=f"strategy-qa-sots-reasoning-path.pkl")

In [ ]:
df = ds.to_pandas()
df.head(10)

In [ ]:
import matplotlib.pyplot as plt

# Tokens distribution
plt.figure()
df['tokens'].hist()
plt.title("Tokens per Example")
plt.xlabel("tokens")
plt.ylabel("count")
plt.show()